Example Python Jupyter Notebook that demonstrates:
- Connecting to a Microsoft SQL Server
- Web scraping data from different pages in a website within a loop
- Cleaning scraped data to get only the information required

The problem:
Lego sets only identify which minifigures are included on the box and not in the instructions. Third party websites exist which include minifigure breakdowns of the sets based on their Lego ID numbers. Using a list of sets owned, it is possible to scrape such websites to collate a list of all the owned minifigures in a particular collection.

In [1]:
# Load SQL Magic
%load_ext sql

In [2]:
# Import required modules
import pandas as pd
import pyodbc
import sqlalchemy
import urllib
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [3]:
# Establish database connection
connection_str = "DRIVER={SQL SERVER};SERVER=LAPTOP-BJU2VQE0\SIMPLESERVER;DATABASE=Lego;TRUSTED_CONNECTION=YES"
connection_str_quoted = urllib.parse.quote_plus(connection_str)
connection_uri = 'mssql+pyodbc:///?odbc_connect={}'.format(connection_str_quoted)

%sql {connection_uri}

In [4]:
filename = "C:\\Users\\Daniel Underwood\\Documents\\HPAlgo\\sets.csv"
sets_df = pd.read_csv(filename)
sets_df.head()

,Set_Number,Set_Name,Quantity
0,21325,Medieval Blacksmith,1
1,31120,Medieval Castle,1
2,40419,Hogwarts Students Acc. Set,2
3,75948,Hogwarts Clock Tower,2
4,75955,Hogwarts Express,1


In [5]:
url = "https://www.bricklink.com/catalogItemInv.asp?S=<set_no>-1&viewItemType=M"
#User-agent headers
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

web_data = pd.DataFrame(columns=["Figure_ID","Figure_Name","Quantity","Set_Number"])
ignore_rows = ['Regular Items:','Minifigures:','Alernative Items:','Description']

for i in range(0,len(sets_df)):

    set_no = sets_df.iloc[i]['Set_Number']

    #replace the <set_no> text with lego set number in the url
    new_url = url.replace("<set_no>",str(set_no))
        
    # Get the contents of the webpage in text format and store in a variable called data
    data = requests.get(new_url, headers=headers).text

    # Parse using BeautifulSoup to make the data more manageable
    soup = BeautifulSoup(data,"html.parser")

    # Find all the tables in the page
    tables = soup.find_all('table')

    #Find the table with the minifigure information in
    for index,td in enumerate(tables):
        if ("Image" in str(td)):
            table_index = index
    
    for row in tables[table_index].find_all("tr"):
        col = row.find_all("td")
        if (col != [] and col[0].text.strip() not in ignore_rows and len(col) >=5):
            if col[3].text.strip() != "Description":

                raw_id = col[2].text.strip()
                raw_id_pos = raw_id.find(' (Inv)')            
                if raw_id_pos == -1:
                    figureid = raw_id
                else:
                    figureid = raw_id[0:raw_id_pos]    

                raw_name = col[3].text.strip().replace(",","")
                raw_name_pos = raw_name.find('Catalog')
                if raw_name_pos == -1:
                    name = raw_name
                else:
                    name = raw_name[0:raw_name.find('Catalog')]
                
                if name[len(name)-1] == " ":
                    name = name[0:len(name)-1]

                quantity = int(col[1].text.strip())            

                new_web_data = pd.DataFrame({"Figure_ID":figureid,"Figure_Name":name,"Quantity":quantity,"Set_Number":int(set_no)},index = [0])
                web_data = pd.concat([web_data,new_web_data],axis=0)

web_data.set_index("Figure_ID",inplace=True)
web_data.to_csv("minifigures.csv")
web_data.head()

,Figure_Name,Quantity,Set_Number
Figure_ID,,,
idea084,Black Falcon - Female Flat Silver Shoulder Pads,1,21325
idea085,Black Falcon - Male Flat Silver Shoulder Pad,1,21325
idea082,Blacksmith Reddish Brown Apron,1,21325
idea083,Huntress Green Tunic,1,21325
cas555,Black Falcon - Castle Guard Female Flat Silver...,1,31120


In [6]:
#Need to drop the minifigures table first as it references the sets table
%sql DROP TABLE IF EXISTS minifigures

 * mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BSQL+SERVER%7D%3BSERVER%3DLAPTOP-BJU2VQE0%5CSIMPLESERVER%3BDATABASE%3DLego%3BTRUSTED_CONNECTION%3DYES
Done.


[]

In [7]:
%%sql
DROP TABLE IF EXISTS sets

CREATE TABLE sets
    (
    Set_Number INT PRIMARY KEY,
    Set_Name VARCHAR(100) NOT NULL,
    Quantity INT NOT NULL
    )

 * mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BSQL+SERVER%7D%3BSERVER%3DLAPTOP-BJU2VQE0%5CSIMPLESERVER%3BDATABASE%3DLego%3BTRUSTED_CONNECTION%3DYES
Done.


[]

In [8]:
%%sql
BULK INSERT sets
FROM "C:\Users\Daniel Underwood\Documents\HPAlgo\sets.csv"
WITH (FORMAT = 'CSV'
      , FIRSTROW = 2
      , FIELDQUOTE = '\'
      , FIELDTERMINATOR = ','
      , ROWTERMINATOR = '0x0a');

 * mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BSQL+SERVER%7D%3BSERVER%3DLAPTOP-BJU2VQE0%5CSIMPLESERVER%3BDATABASE%3DLego%3BTRUSTED_CONNECTION%3DYES
13 rows affected.


[]

In [9]:
%sql select * from sets

 * mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BSQL+SERVER%7D%3BSERVER%3DLAPTOP-BJU2VQE0%5CSIMPLESERVER%3BDATABASE%3DLego%3BTRUSTED_CONNECTION%3DYES
Done.


Set_Number,Set_Name,Quantity
21325,Medieval Blacksmith,1
31120,Medieval Castle,1
40419,Hogwarts Students Acc. Set,2
75948,Hogwarts Clock Tower,2
75955,Hogwarts Express,1
75968,4 Privet Drive,1
75969,Hogwarts Astronomy Tower,2
75980,Attack on the Burrow,1
76388,Hogsmeade Village Visit,1
76389,Hogwarts Chamber of Secrets,2


In [10]:
%%sql

CREATE TABLE minifigures
    (
    Figure_ID VARCHAR(8) PRIMARY KEY,
    Figure_Name VARCHAR(125) NOT NULL,
    Quantity INT NOT NULL,
    Set_Number INT FOREIGN KEY REFERENCES sets(Set_Number) NOT NULL    
    )

 * mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BSQL+SERVER%7D%3BSERVER%3DLAPTOP-BJU2VQE0%5CSIMPLESERVER%3BDATABASE%3DLego%3BTRUSTED_CONNECTION%3DYES
Done.


[]

In [11]:
%%sql
BULK INSERT minifigures
FROM "C:\Users\Daniel Underwood\Documents\HPAlgo\minifigures.csv"
WITH (FORMAT = 'CSV'
      , FIRSTROW = 2
      , FIELDQUOTE = '\'
      , FIELDTERMINATOR = ','
      , ROWTERMINATOR = '0x0a');

 * mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BSQL+SERVER%7D%3BSERVER%3DLAPTOP-BJU2VQE0%5CSIMPLESERVER%3BDATABASE%3DLego%3BTRUSTED_CONNECTION%3DYES
75 rows affected.


[]

The next section details a series of sql queries against the newly created database information. The Lego sets in this collection are mostly from the Harry Potter series. Most of these sets include a Harry Potter minifigure, so it can be interesting to determine some statistics based on this. The queries cover:
- Total number of minifigures
- Total number of Harry Potter figures
- Details of the different Harry Potter figures
- List of sets that don't have a Harry Potter figure

In [12]:
%%sql
select sum(f.Quantity*s.Quantity) as 'Total Minifigures' from minifigures as f
inner join sets as s on s.Set_Number = f.Set_Number

 * mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BSQL+SERVER%7D%3BSERVER%3DLAPTOP-BJU2VQE0%5CSIMPLESERVER%3BDATABASE%3DLego%3BTRUSTED_CONNECTION%3DYES
Done.


Total Minifigures
106


In [13]:
%%sql
select sum(f.Quantity*s.Quantity) as 'Total Harrys' from minifigures as f
inner join sets as s on s.Set_Number = f.Set_Number where f.Figure_Name like '%Harry Potter%'

 * mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BSQL+SERVER%7D%3BSERVER%3DLAPTOP-BJU2VQE0%5CSIMPLESERVER%3BDATABASE%3DLego%3BTRUSTED_CONNECTION%3DYES
Done.


Total Harrys
14


In [14]:
%%sql
select f.Figure_Name, s.Set_Name, f.Quantity*s.Quantity as Total from minifigures as f
inner join sets as s on s.Set_Number = f.Set_Number where f.Figure_Name like '%Harry Potter%'
group by f.Figure_Name, s.Set_Name, f.Quantity, s.Quantity

 * mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BSQL+SERVER%7D%3BSERVER%3DLAPTOP-BJU2VQE0%5CSIMPLESERVER%3BDATABASE%3DLego%3BTRUSTED_CONNECTION%3DYES
Done.


Figure_Name,Set_Name,Total
Harry Potter Black Suit White Bow Tie,Hogwarts Clock Tower,2
Harry Potter Dark Blue Open Jacket Black Medium Legs,Hogsmeade Village Visit,1
Harry Potter Dark Blue Open Jacket with Tears and Blood Stains Dark Tan Medium Legs Smile / Angry Mouth,Hogwarts Hospital Wing,1
Harry Potter Dark Blue Zip Up,Hogwarts Express,1
Harry Potter Dark Red Plaid Flannel Shirt Black Short Legs,4 Privet Drive,1
Harry Potter Dark Red Shirt and Tie Black Robe,Hogwarts Astronomy Tower,2
Harry Potter Dark Red Torn Sweater,Hogwarts Wizard's Chess,1
Harry Potter Gryffindor Robe Open Sweater Shirt and Tie Black Short Legs,Hogwarts Chamber of Secrets,2
Harry Potter Gryffindor Sweater Black Legs,Hogwarts Students Acc. Set,2
Harry Potter Light Bluish Gray Hooded Sweatshirt,Attack on the Burrow,1


In [15]:
%%sql
select s.Set_Number, s.Set_Name from sets as s where s.Set_Number NOT IN
(select f.Set_Number from minifigures as f where f.Figure_Name like '%Harry Potter%')

 * mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BSQL+SERVER%7D%3BSERVER%3DLAPTOP-BJU2VQE0%5CSIMPLESERVER%3BDATABASE%3DLego%3BTRUSTED_CONNECTION%3DYES
Done.


Set_Number,Set_Name
21325,Medieval Blacksmith
31120,Medieval Castle
76399,Hogwarts Magical Trunk
